In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from pathlib import Path
import json

# Step 1: Define paths and hyperparameters
DATA_ROOT = "/content/drive/MyDrive/NTU-Roselab-Dataset"
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 8
LEARNING_RATE = 1e-4
N_FOLDS = 5
MODEL_NAME = "Hybrid_MobileNetV2"
OUTPUT_DIR = f"/content/drive/MyDrive/Recapture_Photo_Detection/{MODEL_NAME}/results"
SPLIT_DIR = "/content/drive/MyDrive/Recapture_Photo_Detection"
CHECKPOINT_FILE = f"{OUTPUT_DIR}/checkpoint.json"
PREPROCESSING = "Fourier_Laplacian"
HYPERPARAMETERS = {
    "learning_rate": LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "optimizer": "Adam",
    "epochs": EPOCHS,
    "n_folds": N_FOLDS,
    "dropout_rate": 0.4
}

# Step 2: Mount Google Drive and verify dataset path
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ImportError:
    raise ImportError("This script must be run in Google Colab with Google Drive mounted.")
if not os.path.exists(DATA_ROOT):
    raise FileNotFoundError(f"Dataset directory {DATA_ROOT} does not exist. Please check the path.")

# Step 3: Check dataset balance
def check_dataset_balance(data_root):
    originals_path = os.path.join(data_root, 'originals')
    recaptures_path = os.path.join(data_root, 'recaptures')
    originals_count = sum(len(files) for _, _, files in os.walk(originals_path))
    recaptures_count = sum(len(files) for _, _, files in os.walk(recaptures_path))
    print(f"Dataset Balance: {originals_count} originals, {recaptures_count} recaptures")
    return originals_count, recaptures_count

originals_count, recaptures_count = check_dataset_balance(DATA_ROOT)

# Step 4: Define Hann window function
@tf.function
def hann2d(h, w):
    hann1 = tf.signal.hann_window(h, periodic=True)
    hann2 = tf.signal.hann_window(w, periodic=True)
    return tf.sqrt(tf.tensordot(hann1, hann2, axes=0))

# Step 5: Define hybrid preprocessing function
@tf.function
def hybrid_preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    gray = tf.image.rgb_to_grayscale(img)
    gray = tf.cast(gray, tf.float32)

    # FFT branch
    fft_img = gray - tf.reduce_mean(gray)
    hann_window = hann2d(IMG_SIZE, IMG_SIZE)
    fft_img *= hann_window
    fft = tf.signal.fft2d(tf.complex(fft_img[..., 0], 0.0))
    fft = tf.signal.fftshift(fft)
    spectrum = tf.math.log1p(tf.abs(fft))
    spectrum = (spectrum - tf.reduce_mean(spectrum)) / (tf.math.reduce_std(spectrum) + 1e-6)
    spectrum = (spectrum - tf.reduce_min(spectrum)) / (tf.reduce_max(spectrum) - tf.reduce_min(spectrum) + 1e-6)
    fft_stack = tf.stack([spectrum, spectrum, spectrum], axis=-1)

    # Laplacian branch
    gray_n = (gray - tf.reduce_min(gray)) / (tf.reduce_max(gray) - tf.reduce_min(gray) + 1e-6)
    lap = tf.image.sobel_edges(gray)  # Shape: (batch, h, w, 1, 2)
    lap = tf.sqrt(tf.reduce_sum(tf.square(lap), axis=-1))  # Shape: (batch, h, w, 1)
    lap = tf.squeeze(lap, axis=-1)  # Shape: (batch, h, w)
    lap = (lap - tf.reduce_mean(lap)) / (tf.math.reduce_std(lap) + 1e-6)
    lap = (lap - tf.reduce_min(lap)) / (tf.reduce_max(lap) - tf.reduce_min(lap) + 1e-6)
    lap_stack = tf.stack([gray_n[..., 0], lap, lap], axis=-1)  # Shape: (batch, h, w, 3)

    return (fft_stack, lap_stack), label

# Step 6: Define custom rotation function
@tf.function
def random_rotation(img, max_angle=0.1):  # max_angle in radians (~10 degrees)
    angles = [0, np.pi/2, np.pi, 3*np.pi/2]  # 0°, 90°, 180°, 270°
    k = tf.random.uniform(shape=(), minval=0, maxval=len(angles), dtype=tf.int32)
    img = tf.image.rot90(img, k)
    return img

# Step 7: Define data augmentation
@tf.function
def augment_image(inputs, label):
    fft_img, lap_img = inputs
    fft_img = tf.image.random_flip_left_right(fft_img)
    lap_img = tf.image.random_flip_left_right(lap_img)
    fft_img = random_rotation(fft_img, max_angle=0.1)
    lap_img = random_rotation(lap_img, max_angle=0.1)
    return (fft_img, lap_img), label

# Step 8: Load or create train-test split
def load_or_create_split():
    if all(os.path.exists(os.path.join(SPLIT_DIR, f)) for f in ['X_train.npy', 'X_test.npy', 'y_train.npy', 'y_test.npy']):
        print("Loading existing train-test split...")
        X_train = np.load(os.path.join(SPLIT_DIR, 'X_train.npy'))
        X_test = np.load(os.path.join(SPLIT_DIR, 'X_test.npy'))
        y_train = np.load(os.path.join(SPLIT_DIR, 'y_train.npy'))
        y_test = np.load(os.path.join(SPLIT_DIR, 'y_test.npy'))
    else:
        dataset = image_dataset_from_directory(
            DATA_ROOT,
            labels='inferred',
            label_mode='binary',
            image_size=(IMG_SIZE, IMG_SIZE),
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=42
        )
        images, labels = [], []
        for img_batch, label_batch in dataset:
            images.append(img_batch.numpy())
            labels.append(label_batch.numpy())
        images = np.concatenate(images, axis=0)
        labels = np.concatenate(labels, axis=0).flatten()

        X_train, X_test, y_train, y_test = train_test_split(
            images, labels, test_size=0.2, stratify=labels, random_state=42
        )

        Path(SPLIT_DIR).mkdir(parents=True, exist_ok=True)
        np.save(os.path.join(SPLIT_DIR, 'X_train.npy'), X_train)
        np.save(os.path.join(SPLIT_DIR, 'X_test.npy'), X_test)
        np.save(os.path.join(SPLIT_DIR, 'y_train.npy'), y_train)
        np.save(os.path.join(SPLIT_DIR, 'y_test.npy'), y_test)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_or_create_split()

# Create test dataset
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE).map(hybrid_preprocess).prefetch(tf.data.AUTOTUNE)

# Step 9: Define function to create two-branch MobileNetV2 model
def create_model():
    fft_in = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='fft_input')
    lap_in = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='lap_input')

    # FFT branch wrapped in Sequential with unique name
    fft_base = Sequential([
        MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    ], name='fft_mobilenetv2')
    for layer in fft_base.layers[0].layers[:-20]:
        layer.trainable = False
    fft_x = GlobalAveragePooling2D()(fft_base(fft_in))

    # Laplacian branch wrapped in Sequential with unique name
    lap_base = Sequential([
        MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    ], name='lap_mobilenetv2')
    for layer in lap_base.layers[0].layers[:-20]:
        layer.trainable = False
    lap_x = GlobalAveragePooling2D()(lap_base(lap_in))

    # Merge branches
    merged = Concatenate()([fft_x, lap_x])
    x = Dropout(HYPERPARAMETERS['dropout_rate'])(merged)
    x = Dense(128, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[fft_in, lap_in], outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Step 10: Convert NumPy types to JSON-serializable types
def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Step 11: Define function to save results
def save_model_results(model, dataset, history, model_name, output_dir, fold=None, preprocessing='None', hyperparameters=None):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    fold_str = f"_fold_{fold}" if fold is not None else ""

    # Evaluate on dataset
    y_true, y_pred = [], []
    for (fft_imgs, lap_imgs), labels in dataset:
        preds = (model.predict([fft_imgs, lap_imgs], verbose=0) > 0.5).astype(int)
        y_true.extend(labels.numpy().astype(int))
        y_pred.extend(preds.flatten())

    # Check prediction distribution
    originals_pred = sum(1 for p in y_pred if p == 0)
    recaptures_pred = sum(1 for p in y_pred if p == 1)
    print(f"Predictions {fold_str}: {originals_pred} originals, {recaptures_pred} recaptures")

    # Classification report
    class_report = classification_report(y_true, y_pred, target_names=['originals', 'recaptured'], output_dict=True)
    class_report_df = pd.DataFrame(class_report).transpose()
    class_report_df.to_csv(f'{output_dir}/{model_name}_classification_report{fold_str}.csv')

    # Confusion matrix (detailed)
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['originals', 'recaptured'], yticklabels=['originals', 'recaptured'])
    plt.title(f'Confusion Matrix - {model_name}{fold_str}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(f'{output_dir}/{model_name}_confusion_matrix{fold_str}.png')
    plt.close()

    # Save confusion matrix as CSV
    cm_df = pd.DataFrame(cm, index=['True_originals', 'True_recaptured'], columns=['Pred_originals', 'Pred_recaptured'])
    cm_df.to_csv(f'{output_dir}/{model_name}_confusion_matrix{fold_str}.csv')

    # Model summary (only for final model)
    if fold is None:
        summary_file = f'{output_dir}/{model_name}_summary.txt'
        with open(summary_file, 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))

    # Calculate total and trainable parameters
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

    # Aggregate results
    results = {
        'Model': model_name,
        'Preprocessing': preprocessing,
        'Accuracy': class_report['accuracy'],
        'Total_Parameters': total_params,
        'Trainable_Parameters': trainable_params,
        'Fold': fold if fold is not None else 'Final'
    }
    if hyperparameters:
        results.update(hyperparameters)
    for label, metrics in class_report.items():
        if isinstance(metrics, dict):
            results.update({
                f'Precision_{label}': metrics['precision'],
                f'Recall_{label}': metrics['recall'],
                f'F1-Score_{label}': metrics['f1-score'],
                f'Support_{label}': metrics['support']
            })

    # Convert NumPy types to JSON-serializable types
    results = {k: convert_to_serializable(v) for k, v in results.items()}

    # Save results to JSON
    with open(f'{output_dir}/{model_name}_results{fold_str}.json', 'w') as f:
        json.dump(results, f, indent=4)

    # Plot and save accuracy/loss curves
    if history is not None:
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(f'Accuracy Curve - {model_name}{fold_str}')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], label='Train Loss')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title(f'Loss Curve - {model_name}{fold_str}')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/{model_name}_accuracy_loss_curve{fold_str}.png')
        plt.close()

    # Save full model for final model (using .keras format)
    if fold is None:
        model.save(f'{output_dir}/{model_name}_model.keras', overwrite=True)

    return results

# Step 12: Checkpointing and resumption logic
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            checkpoint = json.load(f)
        last_completed_fold = checkpoint.get('last_completed_fold', 0)
        print(f"Resuming from checkpoint: Last completed fold = {last_completed_fold}")
        return last_completed_fold
    return 0

def save_checkpoint(fold):
    Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
    checkpoint = {'last_completed_fold': fold}
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint, f, indent=4)

# Step 13: Perform 5-fold cross-validation with checkpointing
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
fold_results = []
last_completed_fold = load_checkpoint()

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
    if fold <= last_completed_fold:
        print(f"Skipping fold {fold} (already completed)")
        # Load results from previous run
        fold_str = f"_fold_{fold}"
        result_file = f'{OUTPUT_DIR}/{MODEL_NAME}_results{fold_str}.json'
        if os.path.exists(result_file):
            with open(result_file, 'r') as f:
                fold_results.append(json.load(f))
        continue

    print(f"\nTraining Fold {fold}/{N_FOLDS}")

    # Create train and validation datasets for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

    train_ds = tf.data.Dataset.from_tensor_slices((X_fold_train, y_fold_train)).batch(BATCH_SIZE).map(hybrid_preprocess).map(augment_image).prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices((X_fold_val, y_fold_val)).batch(BATCH_SIZE).map(hybrid_preprocess).prefetch(tf.data.AUTOTUNE)

    # Create model and load weights if available
    model = create_model()
    fold_str = f"_fold_{fold}"
    checkpoint_path = f'{OUTPUT_DIR}/{MODEL_NAME}_model{fold_str}.weights.h5'
    if os.path.exists(checkpoint_path):
        print(f"Loading weights for fold {fold} from {checkpoint_path}")
        model.load_weights(checkpoint_path)

    # Train model with early stopping and checkpointing
    checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, save_weights_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, class_weight={0: 1.0, 1: 1.5}, verbose=1, callbacks=[early_stopping, checkpoint_callback])

    # Save results for this fold
    results = save_model_results(
        model, val_ds, history, MODEL_NAME, OUTPUT_DIR,
        fold=fold, preprocessing=PREPROCESSING, hyperparameters=HYPERPARAMETERS
    )
    fold_results.append(results)

    # Update checkpoint
    save_checkpoint(fold)

# Step 14: Train final model on full training set if not already done
if last_completed_fold < N_FOLDS + 1:
    print("\nTraining final model on full training set")
    final_model_path = f'{OUTPUT_DIR}/{MODEL_NAME}_model.keras'
    if os.path.exists(final_model_path) and last_completed_fold == 'final':
        print(f"Final model already saved at {final_model_path}, skipping training")
        # Load results from previous run
        result_file = f'{OUTPUT_DIR}/{MODEL_NAME}_results.json'
        if os.path.exists(result_file):
            with open(result_file, 'r') as f:
                results = json.load(f)
            print(f"Final Results for {MODEL_NAME}:", results)
    else:
        train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).map(hybrid_preprocess).map(augment_image).prefetch(tf.data.AUTOTUNE)
        model = create_model()
        checkpoint_path = f'{OUTPUT_DIR}/{MODEL_NAME}_model.weights.h5'
        if os.path.exists(checkpoint_path):
            print(f"Loading weights for final model from {checkpoint_path}")
            model.load_weights(checkpoint_path)

        checkpoint_callback = ModelCheckpoint(checkpoint_path, monitor='loss', save_best_only=True, save_weights_only=True)
        history = model.fit(train_ds, epochs=EPOCHS, class_weight={0: 1.0, 1: 1.5}, verbose=1, callbacks=[checkpoint_callback])

        # Save results for final model
        results = save_model_results(
            model, test_ds, history, MODEL_NAME, OUTPUT_DIR,
            preprocessing=PREPROCESSING, hyperparameters=HYPERPARAMETERS
        )
        print(f"Final Results for {MODEL_NAME}:", results)

        # Update checkpoint
        save_checkpoint('final')

# Step 15: Aggregate cross-validation results
if fold_results:
    fold_df = pd.DataFrame(fold_results)
    mean_results = {
        'Model': MODEL_NAME,
        'Preprocessing': PREPROCESSING,
        'Mean_Accuracy': fold_df['Accuracy'].mean(),
        'Std_Accuracy': fold_df['Accuracy'].std(),
        'Mean_Precision_recaptured': fold_df['Precision_recaptured'].mean(),
        'Mean_Recall_recaptured': fold_df['Recall_recaptured'].mean(),
        'Mean_F1-Score_recaptured': fold_df['F1-Score_recaptured'].mean(),
        'Mean_Total_Parameters': fold_df['Total_Parameters'].mean(),
        'Mean_Trainable_Parameters': fold_df['Trainable_Parameters'].mean()
    }
    # Convert NumPy types in mean_results
    mean_results = {k: convert_to_serializable(v) for k, v in mean_results.items()}
    with open(f'{OUTPUT_DIR}/{MODEL_NAME}_cv_summary.json', 'w') as f:
        json.dump(mean_results, f, indent=4)
    fold_df.to_csv(f'{OUTPUT_DIR}/{MODEL_NAME}_cv_results.csv', index=False)
    print("\nCross-Validation Summary:", mean_results)

Mounted at /content/drive
Dataset Balance: 1202 originals, 1199 recaptures
Loading existing train-test split...
Resuming from checkpoint: Last completed fold = 5
Skipping fold 1 (already completed)
Skipping fold 2 (already completed)
Skipping fold 3 (already completed)
Skipping fold 4 (already completed)
Skipping fold 5 (already completed)

Training final model on full training set
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Loading weights for final model from /content/drive/MyDrive/Recapture_Photo_Detection/Hybrid_MobileNetV2/results/Hybrid_MobileNetV2_model.weights.h5
Epoch 1/8


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 94 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


120/120 ━━━━━━━━━━━━━━━━━━━━ 295s 2s/step - accuracy: 0.9809 - loss: 0.0732
Epoch 2/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.9897 - loss: 0.0468
Epoch 3/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 292s 2s/step - accuracy: 0.9874 - loss: 0.0408
Epoch 4/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 308s 2s/step - accuracy: 0.9910 - loss: 0.0337
Epoch 5/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 306s 3s/step - accuracy: 0.9896 - loss: 0.0399
Epoch 6/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.9873 - loss: 0.0569
Epoch 7/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 283s 2s/step - accuracy: 0.9908 - loss: 0.0306
Epoch 8/8
120/120 ━━━━━━━━━━━━━━━━━━━━ 274s 2s/step - accuracy: 0.9950 - loss: 0.0210
Predictions : 158 originals, 323 recaptures


Final Results for Hybrid_MobileNetV2: {'Model': 'Hybrid_MobileNetV2', 'Preprocessing': 'Fourier_Laplacian', 'Accuracy': 0.8066528066528067, 'Total_Parameters': 4843905, 'Trainable_Parameters': 2740097, 'Fold': 'Final', 'learning_rate': 0.0001, 'batch_size': 16, 'optimizer': 'Adam', 'epochs': 8, 'n_folds': 5, 'dropout_rate': 0.4, 'Precision_originals': 0.9683544303797469, 'Recall_originals': 0.6348547717842323, 'F1-Score_originals': 0.7669172932330827, 'Support_originals': 241.0, 'Precision_recaptured': 0.7275541795665634, 'Recall_recaptured': 0.9791666666666666, 'F1-Score_recaptured': 0.8348134991119005, 'Support_recaptured': 240.0, 'Precision_macro avg': 0.8479543049731552, 'Recall_macro avg': 0.8070107192254494, 'F1-Score_macro avg': 0.8008653961724916, 'Support_macro avg': 481.0, 'Precision_weighted avg': 0.8482046170841875, 'Recall_weighted avg': 0.8066528066528067, 'F1-Score_weighted avg': 0.8007948179959024, 'Support_weighted avg': 481.0}

Cross-Validation Summary: {'Model': 'Hyb